In [4]:
import pandas as pd
from pathlib import Path

In [5]:
here = Path("__file__").parent
data_folder = here / "data"

roster = pd.read_csv(
    data_folder/"roster.csv",
    converters = {
                "NetID": str.lower, 
                "Email Address": str.lower
                },
    usecols = ["Section", "Email Address", "NetID"],
    index_col = "NetID"
)

# dataFrames.
hwcsv = pd.read_csv(
    data_folder/"hw_exam_grades.csv",
    converters = {
                "Email Address": str.lower,
                "SID": str.lower,
                "First Name": str.lower,
                "Last Name": str.lower
    },
    usecols= lambda x: "Submission" not in x,
    index_col = "SID"
)

# [i for i in data_folder.glob("quiz_*_grades.csv")]

quiz_grades = pd.DataFrame()

for file_path in data_folder.glob("quiz_*_grades.csv"):
    quiz_name = " ".join(file_path.stem.title().split("_")[:2])
    # print(file_path)
    quiz = pd.read_csv(
        file_path,
        converters = {
            "Email": str.lower
        },
        usecols=["Email", "Grade"],
        index_col="Email"
    ).rename(columns={"Grade": quiz_name})
    quiz_grades = pd.concat([quiz_grades, quiz], axis=1)
    

In [37]:
finalData = pd.merge(
    roster,
    hwcsv,
    left_index=True,
    right_index=True
)

finalData = pd.merge(
    finalData,
    quiz_grades,
    left_on = "Email Address",
    right_index = True
    
)

finalData = finalData.fillna(0)

# Average Exam score

for i in range(1,4):
    finalData[f"Exam {i} Score"] = (finalData[f"Exam {i}"] / finalData[f"Exam {i} - Max Points"])
    
homework_score = finalData.filter(regex=r"^Homework \d\d?$", axis = 1)

homework_score


,Homework 1,Homework 2,Homework 3,Homework 4,Homework 5,Homework 6,Homework 7,Homework 8,Homework 9,Homework 10
wxb12345,55.0,62,73,83,68,87,36,43,45,46
mxl12345,63.0,57,78,83,61,88,42,58,45,37
txj12345,0.0,77,58,91,66,78,30,69,59,53
jgf12345,69.0,52,64,75,61,83,41,43,42,40
smj00936,74.0,55,60,67,46,85,43,42,59,48
...,...,...,...,...,...,...,...,...,...,...
pmj37756,73.0,50,55,65,47,76,32,55,40,51
dsl24347,69.0,51,70,96,51,77,37,52,49,57
nxe44872,62.0,76,62,79,64,74,37,58,58,51
bxr62103,53.0,50,55,97,57,75,39,64,41,46
